In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import string
import re

import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import gensim
import spacy 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\REEEM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
df = pd.read_excel("season.xlsx")
df.head(16)

,conversation_id,created_at,favorite_count,full_text,hashtags/0,hashtags/1,hashtags/2,hashtags/3,hashtags/4,hashtags/5,...,user_mentions/14/screen_name,user_mentions/15/id_str,user_mentions/15/name,user_mentions/15/screen_name,user_mentions/16/id_str,user_mentions/16/name,user_mentions/16/screen_name,user_mentions/17/id_str,user_mentions/17/name,user_mentions/17/screen_name
0,1474707073342394370,2021-12-25T11:42:19.000Z,24,سولت محبوب الكل الآن في #بوليفارد_رياض_سيتي \n...,بوليفارد_رياض_سيتي,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1474688020070780937,2021-12-25T10:26:37.000Z,27,جدول اليوم في #موسم_الرياض \nرتب يومك .. واحجز...,موسم_الرياض,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1474537577676001283,2021-12-25T00:28:48.000Z,60,ليلة الكلّ بيتذكرها .. من زود جمالها ❤️\nعشناه...,بوليفارد_رياض_سيتي,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1474510457771675649,2021-12-24T22:41:02.000Z,68,مع عمالقة الفن العراقي \nالفنان القدير حاتم ال...,بوليفارد_رياض_سيتي,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1474508006465236995,2021-12-24T22:31:18.000Z,35,عشان المغامرات ما توقف .. والتجارب الحلوة تستم...,رياض_سفاري,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1474501540572393473,2021-12-24T22:05:36.000Z,61,وحدة من الليالي الطربية \nبنعيشها مع الفنانة ه...,ذا_جروفز,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1474501141287165953,2021-12-24T22:04:01.000Z,26,بأصالة الطرب الشرقي \nالفنانة مروة ناجي بتسمِّ...,ذا_جروفز,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1474500686473691137,2021-12-24T22:02:13.000Z,47,على مسرح من جليد ❄️\nبنعيش تجربة بصرية مذهلة و...,بوليفارد_رياض_سيتي,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1474456118017486849,2021-12-24T19:05:07.000Z,47,نجوم السَّاحة الطربية .. جايين يطربونا بأعذب أ...,اوايسس,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1474454487687319552,2021-12-24T18:58:38.000Z,62,لكل اللي يبون مسرحية تسعد أطفالهم ‍‍‍\nبدر وأل...,بوليفارد_رياض_سيتي,موسم_الرياض,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df = df[['created_at', 'favorite_count', 'full_text']]

In [23]:
df

,created_at,favorite_count,full_text
0,2021-12-25T11:42:19.000Z,24,سولت محبوب الكل الآن في #بوليفارد_رياض_سيتي \n...
1,2021-12-25T10:26:37.000Z,27,جدول اليوم في #موسم_الرياض \nرتب يومك .. واحجز...
2,2021-12-25T00:28:48.000Z,60,ليلة الكلّ بيتذكرها .. من زود جمالها ❤️\nعشناه...
3,2021-12-24T22:41:02.000Z,68,مع عمالقة الفن العراقي \nالفنان القدير حاتم ال...
4,2021-12-24T22:31:18.000Z,35,عشان المغامرات ما توقف .. والتجارب الحلوة تستم...
...,...,...,...
964,2019-09-03T09:04:42.000Z,2,I discovered @apify a few months ago and start...
965,2021-09-20T09:02:48.000Z,5,@evapavlikova Abroad definitely @samjulien (wh...
966,2021-08-17T00:50:29.000Z,3,@bardeenai @apify @airtable This is awesome ma...
967,2019-04-08T16:21:54.000Z,4,@jancurn @jakubbalada @apify Such a long post!...


In [24]:
def remove_hashtag(df, col = 'full_text'):
    for letter in r'#.][!XR':
        df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
remove_hashtag(df)
df

<ipython-input-24-10f5318cd9d7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(str).str.replace(letter,'', regex=True)


,created_at,favorite_count,full_text
0,2021-12-25T11:42:19.000Z,24,سولت محبوب الكل الآن في بوليفارد_رياض_سيتي \nو...
1,2021-12-25T10:26:37.000Z,27,جدول اليوم في موسم_الرياض \nرتب يومك واحجز تذ...
2,2021-12-25T00:28:48.000Z,60,ليلة الكلّ بيتذكرها من زود جمالها ❤️\nعشناها ...
3,2021-12-24T22:41:02.000Z,68,مع عمالقة الفن العراقي \nالفنان القدير حاتم ال...
4,2021-12-24T22:31:18.000Z,35,عشان المغامرات ما توقف والتجارب الحلوة تستمر ...
...,...,...,...
964,2019-09-03T09:04:42.000Z,2,I discovered @apify a few months ago and start...
965,2021-09-20T09:02:48.000Z,5,@evapavlikova Abroad definitely @samjulien (wh...
966,2021-08-17T00:50:29.000Z,3,@bardeenai @apify @airtable This is awesome ma...
967,2019-04-08T16:21:54.000Z,4,@jancurn @jakubbalada @apify Such a long post ...


In [31]:
text = df['full_text']

In [36]:
arabic_punctuation = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
english_punctuation= string.punctuation
punctuation_list = arabic_punctuation + english_punctuation

def remove_punctuation(text):
    translator = str.maketrans('', '', punctuation_list)
    return text.translate(translator)

In [37]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

In [38]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1',text)

In [40]:
def processDocument(doc, stemmer):
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', ' ', doc)
    doc = re.sub(r'\d', ' ', doc)
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ', doc)
    doc = re.sub(r'#([^\s]+)',r'\1', doc)
    doc = remove_punctuation(doc)
    doc = normalize_arabic(doc)
    doc = remove_repeating_char(doc)
    doc = stemmer.stem(doc)
    
    return doc

In [25]:
# adapted from https://github.com/bakrianoo/aravec
# function to clean and normalize text 
def clean_text(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']  
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
        
    text = text.strip()
    
    return text


In [44]:
stemmer = ISRIStemmer()
df['full_text'] = df['full_text'].apply(lambda x: processDocument(x, stemmer))
df['full_text'].unique()

<ipython-input-44-ac7d45580280>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'] = df['full_text'].apply(lambda x: processDocument(x, stemmer))


array(['سولت محبوب الكل الان في بوليفارد رياض سيتي وبجلسات تناسب الاجواء الحلوه استمتع بالجو الرهيب والاكل الذيذ ❤️ تعال ولا تفوتها موسم الرياض ',
       'جدول اليوم في موسم الرياض رتب يومك واحجز تذكرتك الحين ❤️ ولا تنسي تلتزم بالاجراءات الاحترازيه ',
       'ليله الكل بيتذكرها من زود جمالها ❤️ عشناها مع كبار مطربين العراق حاتم العراقي واصيل هميم وسيف نبيل بليله دجله والفرات في بوليفارد رياض سيتي موسم الرياض ',
       'مع عمالقه الفن العراقي الفنان القدير حاتم العراقي والفنانه اصيل هميم والفنان سيف نبيل يطربون ليلتنا بمحمد عبده ارينا في بوليفارد رياض سيتي موسم الرياض ',
       'عشان المغامرات ما توقف والتجارب الحلوه تستمر ❤️ منطقه رياض سفاري تستقبلكم الي يوم يناير احجز تذكرتك الحين ولا تنسي الالتزام بالاجراءات الاحترازيه موسم الرياض ',
       'وحده من اليالي الطربيه بنعيشها مع الفنانه هايدي موسي في منطقه ذا جروفز يوم ديسمبر ✨ لا تفوتك واحجز طاولتك الحين ❤️ موسم الرياض ',
       'باصاله الطرب الشرقي الفنانه مروه ناجي بتسمعنا احلي الاغاني يوم ديسمبر في منطقه ذا جروفز اطرب معنا احجز طاولت

In [26]:
# adapted from https://github.com/bakrianoo/aravec


# Load AraVec model from gensim    
model = gensim.models.Word2Vec.load("full_grams_cbow_100_twitter.mdl")

# export to word2vec format for spacy model
model.wv.save_word2vec_format("./spacyModel/aravec.txt")
# run the following command gzip ./spacyModel/aravec.txt   
# run the following command python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz
 
# load spacy model 
nlp = spacy.load("./spacy.aravec.model/")

FileNotFoundError: [Errno 2] No such file or directory: 'full_grams_cbow_100_twitter.mdl'